In [1]:
# Importando Bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import math
import scipy.stats as stats
from tqdm import tqdm 
import time
import mplcyberpunk
import smtplib

# Cálculo Numérico
### Ferramenta auxiliar

*Thiago Laidler Vidal Cunha*

### 1. Resolução de Sistemas Lineares por métodos diretos
#### 1.1 - Eliminação de Gauss

In [2]:
def eliminacao_de_gauss(A, b):
    n = len(b)
    
    # Eliminação
    for i in range(n-1):
        for j in range(i+1, n):
            if A[j, i] != 0:
                fator = A[j, i] / A[i, i]
                A[j, i+1:] -= fator * A[i, i+1:]
                b[j] -= fator * b[i]
    # Substituição
    x = np.zeros(n)
    x[n-1] = b[n-1] / A[n-1, n-1]
    
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i] #np.dot multiplica matrizes
    
    return x

# Exemplo de sistema linear
A = np.array([[2.0, 3.0, -1.0],
              [4.0, 4.0, -3.0],
              [2.0, -3.0, 1.0]])

b = np.array([4.0, 2.0, -1.0])

# Chama a função de eliminação de Gauss para resolver o sistema
solucao = eliminacao_de_gauss(A, b)

print("Solução do sistema:")
print(solucao)

Solução do sistema:
[0.75 1.7  2.6 ]


In [3]:
'''
Resolvendo exemplo 1 em aula
'''

A = np.array([[1.0, 2.0, -1.0, 1.0],
              [-1.0, 1.0, 2.0, 0.0],
              [2.0, 0.0, 1.0, 2.0],
              [1.0, 1.0, -3.0, 2.0]])
b = np.array([0.0, 1.0, 3.0,-2.0])

eliminacao_de_gauss(A,b)

array([1., 0., 1., 0.])

In [4]:
def eliminacao_de_gauss(A, b):
    '''
    Resolução com Pivô e guardando a matriz triangular final
    '''
    n = len(b)
    Ab = np.column_stack((A, b))

    for i in range(n-1):
        pivot = Ab[i, i]
        if pivot == 0:
            raise ValueError("O pivô é zero, o sistema não possui solução única.")
        
        Ab[i, :] /= pivot

        for j in range(i+1, n):
            multiplier = Ab[j, i] / Ab[i, i]
            Ab[j, :] -= multiplier * Ab[i, :]
    
    Ab[-1, :] /= Ab[-1, -2]  # Divisão para normalizar o último elemento

    df = pd.DataFrame(Ab[:, :-1], columns=[f'x{i+1}' for i in range(n)])
    df['b'] = Ab[:, -1]
    
    return df

In [5]:
A = np.array([[1.0, 2.0, -1.0, 1.0],
              [-1.0, 1.0, 2.0, 0.0],
              [2.0, 0.0, 1.0, 2.0],
              [1.0, 1.0, -3.0, 2.0]])
b = np.array([0.0, 1.0, 3.0,-2.0])

eliminacao_de_gauss(A,b)

,x1,x2,x3,x4,b
0,1.0,2.0,-1.000000,1.000000,0.000000
1,0.0,1.0,0.333333,0.333333,0.333333
2,0.0,0.0,1.000000,0.307692,1.000000
3,0.0,0.0,0.000000,1.000000,0.000000


#### 1.2 - Fatoração LU

In [6]:
import numpy as np

def fatoracao_LU(A, b):
    n = len(b)
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for k in range(n):
        U[k, k:] = A[k, k:] - np.dot(L[k, :k], U[:k, k:])
        L[(k+1):, k] = (A[(k+1):, k] - np.dot(L[(k+1):, :k], U[:k, k])) / U[k, k]
    
    # Resolvendo Ly = b por substituição progressiva
    y = np.zeros(n)
    y[0] = b[0]
    
    for i in range(1, n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    # Resolvendo Ux = y por substituição regressiva
    x = np.zeros(n)
    x[n-1] = y[n-1] / U[n-1, n-1]
    
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    
    return x

# Exemplo de sistema linear
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]])

b = np.array([8, -11, -3])

# Chama a função de fatoração LU para resolver o sistema
solucao = fatoracao_LU(A, b)

print("Solução do sistema:")
print(solucao)

Solução do sistema:
[ 2.  3. -1.]


### 2. Resolução de Sistemas Lineares por métodos iterativos
#### 2.1 - Gauss-Jacobi

In [7]:
def gauss_jacobi(A, b, tolerance=0.05):
    n = len(A)
    x = np.zeros(n)
    x_prev = np.zeros(n)
    iterations = []
    
    dk_relativo = tolerance + 1
    k = 0
    
    while dk_relativo > tolerance:
        dx_max = 0
        
        for i in range(n):
            temp_sum = 0
            
            for j in range(n):
                if j != i:
                    temp_sum += A[i][j] * x_prev[j]
            
            x[i] = (b[i] - temp_sum) / A[i][i]
            
            dx = abs(x[i] - x_prev[i])
            if dx > dx_max:
                dx_max = dx
        
        dk_relativo = dx_max / max(abs(x))
        iterations.append([k+1] + list(x) + [dx_max, dk_relativo])
        
        x_prev = x.copy()
        k += 1
    
    column_names = ['k'] + [f'x{i+1}' for i in range(n)] + ['dk', 'dk_relativo']
    df = pd.DataFrame(iterations, columns=column_names)
    return df

In [8]:
# Exemplo de sistema linear
A = np.array([[4, 1, 2],
              [-1, 3, 1],
              [2, 2, 5]])
b = np.array([20, 30, 65])

# Chama a função para resolver o sistema pelo método de Gauss-Jacobi
result = gauss_jacobi(A, b,0.05)
result

,k,x1,x2,x3,dk,dk_relativo
0,1,5.000000,10.000000,13.000000,13.000000,1.000000
1,2,-4.000000,7.333333,7.000000,9.000000,1.227273
2,3,-0.333333,6.333333,11.666667,4.666667,0.400000
3,4,-2.416667,6.000000,10.600000,2.083333,0.196541
4,5,-1.800000,5.661111,11.566667,0.966667,0.083573
5,6,-2.198611,5.544444,11.455556,0.398611,0.034796


#### 2.2 - Gauss-Seidel

In [9]:
def gauss_seidel(A, b, tolerance=0.05):
    n = len(A)
    x = np.zeros(n)
    iterations = []
    
    dk_relativo = tolerance + 1
    k = 0
    
    while dk_relativo > tolerance:
        dx_max = 0
        
        for i in range(n):
            x_prev = x.copy()
            temp_sum = 0
            
            for j in range(n):
                if j != i:
                    temp_sum += A[i][j] * x[j]
            
            x[i] = (b[i] - temp_sum) / A[i][i]
            
            dx = abs(x[i] - x_prev[i])
            if dx > dx_max:
                dx_max = dx
        
        dk_relativo = dx_max / max(abs(x))
        iterations.append([k+1] + list(x) + [dx_max, dk_relativo])
        
        k += 1
    
    column_names = ['k'] + [f'x{i+1}' for i in range(n)] + ['dk', 'dk_relativo']
    df = pd.DataFrame(iterations, columns=column_names)
    return df

In [10]:
# Exemplo de sistema linear
A = np.array([[4.0, -1.0, 0.0],
              [-1.0, 4.0, -1.0],
              [0.0, -1.0, 4.0]])

b = np.array([2.0, 5.0, 1.0])

# Chama a função de Gauss-Seidel para resolver o sistema
solucao = gauss_seidel(A, b)
solucao

,k,x1,x2,x3,dk,dk_relativo
0,1,0.500000,1.375000,0.593750,1.375000,1.000000
1,2,0.843750,1.609375,0.652344,0.343750,0.213592
2,3,0.902344,1.638672,0.659668,0.058594,0.035757


In [11]:
# Exemplo de sistema linear
A = np.array([[4, -1, 1],
              [1, 3, -2],
              [2, -1, 5]])
b = np.array([5, -8, 14])

# Chama a função para resolver o sistema pelo método de Gauss-Seidel
result = gauss_seidel(A, b,0.05)
result

,k,x1,x2,x3,dk,dk_relativo
0,1,1.250000,-3.083333,1.683333,3.083333,1.000000
1,2,0.058333,-1.563889,2.463889,1.519444,0.616685
2,3,0.243056,-1.105093,2.481759,0.458796,0.184867
3,4,0.353287,-1.129923,2.432701,0.110231,0.045312


### 3. Interpolação

#### 1. Forma Clássica / Vandermonde

In [12]:
def Inter_Classica(x, y):
    n = len(x)
    V = np.vander(x, increasing=True) #gera uma matriz vandermonde cujos elementos são potencias de x
    
    # Resolvendo o sistema linear utilizando o método de eliminação de Gauss 
    for i in range(n):
        pivot = V[i, i]
        for j in range(i+1, n):
            factor = V[j, i] / pivot
            V[j, i:] -= factor * V[i, i:]
            y[j] -= factor * y[i]
    '''
    era possível utilizar a função np.linalg.solve(V,y) 
    '''
    # Obtendo os coeficientes
    coef = np.zeros(n)
    for i in range(n-1, -1, -1):
        coef[i] = y[i] / V[i, i]
        for j in range(i-1, -1, -1):
            y[j] -= V[j, i] * coef[i]
    
    return coef

In [13]:
Inter_Classica([0.0,1.0,2.0,3.0],[1.0,6.0,5.0,-8.0]) 

array([ 1.,  6.,  0., -1.])

In [14]:
'''
versão simplificada:
def Inter_Classica(x, y):
    n = len(x)
    V = np.vander(x, increasing=True)
    coefficients = np.linalg.solve(V, y) #simplesmente utilizando a função linalg do numpy
    return coefficients
'''

'\nversão simplificada:\ndef Inter_Classica(x, y):\n    n = len(x)\n    V = np.vander(x, increasing=True)\n    coefficients = np.linalg.solve(V, y) #simplesmente utilizando a função linalg do numpy\n    return coefficients\n'

In [15]:
'''
#########Ideia de interação com usuário#########

# Obter os dados de entrada do usuário
n = int(input("Digite o número de pontos de dados: "))

x = []
y = []

for i in range(n):
    point = input(f"Digite as coordenadas (x, y) do ponto {i+1}, separadas por espaço: ")
    point = point.split()
    x.append(float(point[0]))
    y.append(float(point[1]))

# Calcular o polinômio interpolador
coefficients = vandermonde_interpolation(x, y)

# Imprimir o polinômio interpolador
print("O polinômio interpolador é dado por:")
print("P(x) =", end=" ")

for i in range(len(coefficients)):
    print(f"({coefficients[i]:.2f})x^{len(coefficients)-i-1}", end=" ")
    if i < len(coefficients) - 1:
        print("+", end=" ")
        
'''

'\n#########Ideia de interação com usuário#########\n\n# Obter os dados de entrada do usuário\nn = int(input("Digite o número de pontos de dados: "))\n\nx = []\ny = []\n\nfor i in range(n):\n    point = input(f"Digite as coordenadas (x, y) do ponto {i+1}, separadas por espaço: ")\n    point = point.split()\n    x.append(float(point[0]))\n    y.append(float(point[1]))\n\n# Calcular o polinômio interpolador\ncoefficients = vandermonde_interpolation(x, y)\n\n# Imprimir o polinômio interpolador\nprint("O polinômio interpolador é dado por:")\nprint("P(x) =", end=" ")\n\nfor i in range(len(coefficients)):\n    print(f"({coefficients[i]:.2f})x^{len(coefficients)-i-1}", end=" ")\n    if i < len(coefficients) - 1:\n        print("+", end=" ")\n        \n'

#### 2. Forma Lagrange

In [16]:
def Inter_Lagrange(x, y):
    n = len(x)
    polyn = np.poly1d(0.0)
    
    for i in range(n):
        # Calcula os coeficientes de Lagrange
        coeficientes = np.poly1d(1.0) # função que gera coeficientes de um polinomio direto 
        for j in range(n):
            if i != j:
                coeficientes *= np.poly1d([1.0, -x[j]]) / (x[i] - x[j])
        
        # Atualiza o polinômio interpolador
        polyn += y[i] * coeficientes
    
    return polyn

In [17]:
'''
##### ideia de interação com usuario####

# Solicita ao usuário os pontos da tabela de dados
n = int(input("Digite o número de pontos na tabela de dados: "))
x = []
y = []
for i in range(n):
    xi = float(input(f"Digite o valor de x{i+1}: "))
    yi = float(input(f"Digite o valor de y{i+1}: "))
    x.append(xi)
    y.append(yi)

# Encontra o polinômio interpolador
interpolating_polynomial = lagrange_interpolation(x, y)

# Imprime o polinômio interpolador
print("Polinômio Interpolador:")
print(interpolating_polynomial)

'''

'\n##### ideia de interação com usuario####\n\n# Solicita ao usuário os pontos da tabela de dados\nn = int(input("Digite o número de pontos na tabela de dados: "))\nx = []\ny = []\nfor i in range(n):\n    xi = float(input(f"Digite o valor de x{i+1}: "))\n    yi = float(input(f"Digite o valor de y{i+1}: "))\n    x.append(xi)\n    y.append(yi)\n\n# Encontra o polinômio interpolador\ninterpolating_polynomial = lagrange_interpolation(x, y)\n\n# Imprime o polinômio interpolador\nprint("Polinômio Interpolador:")\nprint(interpolating_polynomial)\n\n'

In [18]:
Inter_Lagrange([0,1,2,3],[1,6,5,-8])

poly1d([-1.,  0.,  6.,  1.])

#### 3. Forma Newton

In [19]:
def Dif_Div(x, y):
    '''
    monta a tabela de diferenciais divididos
    '''
    n = len(x)
    tabela = np.zeros((n, n))
    tabela[:, 0] = y
    
    for j in range(1, n):
        for i in range(n - j):
            tabela[i, j] = (tabela[i+1, j-1] - tabela[i, j-1]) / (x[i+j] - x[i])
            
    df = pd.DataFrame(tabela, columns=['Or0'] + [f'Or{i}' for i in range(1, n)])
    df.insert(0, 'x', x)
    
    return tabela,df

def newton_interpol(x, y):
    n = len(x)
    coef = Dif_Div(x, y)[0]
    polyn = np.poly1d(coef[0, :])
    
    return polyn

###Exemplo:
x=[0,1,2,3]
y=[1,6,5,-8]
# Encontra o polinômio interpolador
interpol_polyn = newton_interpol(x, y)

# Imprime o polinômio interpolador
print("Polinômio Interpolador:")
print(interpol_polyn)

# Imprime a tabela de diferenças divididas
tabela = Dif_Div(x,y)[1]
tabela

Polinômio Interpolador:
   3     2
1 x + 5 x - 3 x - 1


,x,Or0,Or1,Or2,Or3
0,0,1.0,5.0,-3.0,-1.0
1,1,6.0,-1.0,-6.0,0.0
2,2,5.0,-13.0,0.0,0.0
3,3,-8.0,0.0,0.0,0.0


 - Análise de Erro (Corolário 3):

In [22]:
def newton_interpolation(x, y, point):
    n = len(x)
    coef = Dif_Div(x, y)[0]

    # Calcula o polinômio interpolador
    polyn = np.poly1d(coef[0, :n])

    # Calcula o erro estimado para o ponto extra
    error_estimation = 0.0
    prod = 1.0
    for i in range(n):
        prod *= (point - x[i])
        error_estimation += coef[0, i] * prod

    return polyn, error_estimation

x = [-2,-1,-0,1,2,3]
y = [-2,-5,1,6,5,-8]
point = 1.5
degree = 3

# Verifica se o grau do polinômio é válido
if degree < 0 or degree >= len(x):
    print("Grau inválido! Deve ser maior ou igual a 0 e menor que o número de pontos na tabela.")
    exit()

# Calcula o polinômio interpolador e o erro estimado
polyn, error = newton_interpolation(x, y, point)

# Devolve a tabela de diferenças divididas
differences_table = Dif_Div(x, y)[1]

# Imprime o polinômio interpolador
print("Polinômio Interpolador:")
print(polyn)

# Imprime o erro estimado
print(f"\nErro estimado para x = {point}: {error}")

differences_table

Polinômio Interpolador:
    5     4       3         2
-2 x - 3 x + 4.5 x - 1.667 x + 0.2083 x - 0.05

Erro estimado para x = 1.5: 13.9453125


,x,Or0,Or1,Or2,Or3,Or4,Or5
0,-2,-2.0,-3.0,4.5,-1.666667,0.208333,-0.05
1,-1,-5.0,6.0,-0.5,-0.833333,-0.041667,0.00
2,0,1.0,5.0,-3.0,-1.000000,0.000000,0.00
3,1,6.0,-1.0,-6.0,0.000000,0.000000,0.00
4,2,5.0,-13.0,0.0,0.000000,0.000000,0.00
5,3,-8.0,0.0,0.0,0.000000,0.000000,0.00
